In [34]:
import pandas as pd

#### Загрузка рабочих файлов
df_ass - информация об оценках в тесте  
df_cou - предметы по семестрам  
df_stass - результаты тестов студентов  
df_streg - информация о регистрации студента

In [35]:
df_ass = pd.read_csv('assessments.csv')
df_cou = pd.read_csv('courses.csv')
df_stass = pd.read_csv('studentAssessment.csv')
df_streg = pd.read_csv('studentRegistration.csv')

#### Задание 4
Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3)

In [36]:
#смотрим, есть ли значения NaN в колонке date_registration
df_streg.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [37]:
#выделяем записи с NaN в колонке date_registration в отдельный массив
#смотрим какие встречаются даты отмены регистрации (date_unregistration) в данном массиве
#выявляем, что в большинстве своем отмена регистрации происходит до начала семестра
#в то же время встречаются значения Nan и положительные значения
date_reg_nan = df_streg[df_streg['date_registration'].isna()]
date_reg_nan.date_unregistration.unique()

array([  nan,   -1.,  -33., -126.,  -38., -142., -109., -149.,  -36.,
        -61.,  -24., -168., -156., -152.,  -29.,  -65., -116.,  -40.,
        -75., -196.,  -81., -213.,   12., -163.,    5.,  146., -166.,
         -2.,  -46., -107., -365., -119.,  -88.,  -82.,  -53.,    9.])

In [38]:
#объединяем данный массив с данными о сданных тестах и видим, 
#что несмотря на то что студенты не являются зарегистрированными на курсе
#они сдают тесты и экзамены после начала семестра
date_reg_nan_merge = date_reg_nan.merge(df_stass, on='id_student')
date_reg_nan_merge.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration,id_assessment,date_submitted,is_banked,score
0,CCC,2014B,394791,NaN,-61.0,24291,20,0,72.0
1,CCC,2014B,394791,NaN,-61.0,24292,116,0,72.0
2,CCC,2014B,394791,NaN,-61.0,24293,158,0,65.0
3,CCC,2014B,394791,NaN,-61.0,24294,204,0,69.0
4,CCC,2014B,394791,NaN,-61.0,24295,21,0,44.0


In [39]:
#таким образом предлагается использовать эти данные для анализа оттока студентов с курса 

In [40]:
#определяем курс с самым высоким количеством регистраций
df_streg.groupby('code_module', as_index = False) \
    .agg({'date_registration': 'count'}) \
    .rename(columns={"date_registration": "number_of_reg"}) \
    .sort_values(by=['number_of_reg'], ascending = False).head(3)

,code_module,number_of_reg
1,BBB,7900
5,FFF,7751
3,DDD,6257


In [41]:
#определяем курс с самым высоким оттоком
df_streg.groupby('code_module', as_index = False) \
    .agg({'date_unregistration': 'count'}) \
    .rename(columns={"date_unregistration": "number_of_unreg"}) \
    .sort_values(by=['number_of_unreg'], ascending = False).head(3)

,code_module,number_of_unreg
5,FFF,2380
1,BBB,2377
3,DDD,2235


#### Дополнение после review

In [42]:
#определяем курс с самым высоким количеством регистраций с учетом уникального id студента
df_streg.groupby('code_module', as_index = False) \
    .agg({'date_registration': 'count','id_student': 'nunique'}) \
    .rename(columns={"date_registration": "number_of_reg", "id_student": 'count_of_unique'}) \
    .sort_values(by=['count_of_unique'], ascending = False)

,code_module,number_of_reg,count_of_unique
1,BBB,7900,7692
5,FFF,7751,7397
3,DDD,6257,5848
2,CCC,4426,4251
4,EEE,2932,2859
6,GGG,2534,2525
0,AAA,748,712


In [43]:
#определяем курс с самым высоким количеством отмены регистрации с учетом уникального id студента
df_streg['unreg'] = df_streg.date_unregistration.notna()

In [44]:
df_streg.query('unreg == True') \
    .groupby('code_module', as_index = False) \
    .agg({'date_unregistration': 'count','id_student': 'nunique'}) \
    .rename(columns={"date_unregistration": "number_of_unreg", "id_student": 'count_of_unique'}) \
    .sort_values(by=['count_of_unique'], ascending = False)

,code_module,number_of_unreg,count_of_unique
1,BBB,2377,2314
5,FFF,2380,2249
3,DDD,2235,2065
2,CCC,1947,1858
4,EEE,718,693
6,GGG,289,287
0,AAA,126,116
